# Branching ratios
Checking the branching ratios using the Meijer Hamiltonian

## Setting up

In [88]:
import numpy as np
import pickle
import sys
import sympy
from tqdm.notebook import tqdm

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state, find_state_idx_from_state, find_closest_state

from sympy.physics.wigner import wigner_3j, wigner_6j

def threej_f(j1,j2,j3,m1,m2,m3):
    return complex(wigner_3j(j1,j2,j3,m1,m2,m3))

def sixj_f(j1,j2,j3,j4,j5,j6):
    return complex(wigner_6j(j1,j2,j3,j4,j5,j6))


with open("B_hamiltonians_symbolic_coupled.py",'rb') as f:
    hamiltonians_B = pickle.load(f)
    
with open("TlF_X_state_hamiltonian0to6_2020_03_03.py",'rb') as f:
    hamiltonians_X = pickle.load(f)



def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation

#Generate list of ground states
Jmin_g = 0
Jmax_g = 6 # max J value in Hamiltonian

# QN_g = []

# for J in np.arange(Jmin_g, Jmax_g+1):
#     for m1 in np.arange(-I_Tl,I_Tl+1):
#         for m2 in np.arange(-I_F,I_F+1):
#             mJ = 0-m1-m2
#             if np.abs(mJ) <= J:
#                 QN_g.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,0))
                
QN_g = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2,0)
      for J  in ni_range(0, Jmax_g+1)
      for mJ in ni_range(-J,J+1)
      for m1 in ni_range(-I_Tl,I_Tl+1)
      for m2 in ni_range(-I_F,I_F+1)
     ]
                
Jmin_g = 0
Jmax_g = 6 # max J value in Hamiltonian

QN_g_coupled = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl,0)
      for J  in ni_range(Jmin_g, Jmax_g+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F,F+1) 
     ]
                
#Generate list of excited states
Jmin_e = 1
Jmax_e = 4

QN_e = []

Omegas = [-1, 1]
QN_e = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl,Omega)
      for J  in ni_range(Jmin_e, Jmax_e+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F, F+1)
      for Omega in Omegas
     ]


In [89]:
#Set electric and magnetic fields
Ez = 50
Bz = 0.01

### Eigenstates for B

In [90]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c_Tl'),
    sympy.symbols('c1p_Tl'),
    sympy.symbols('D_TlF'),
    sympy.symbols('mu_B'),
    *sympy.symbols('gS gL')
]

lambdified_hamiltonians_B = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_B.items()
}

Insert the numerical constants:

In [91]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c_Tl = -7.83e6
c_Tl_p = 11.17e6
D_TlF = 2.28 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]
mu_B = 2e6 #Hz/G
gL = 1
gS = 2

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c_Tl,
        c_Tl_p,
        D_TlF,
        mu_B,
        gS, gL
    )
    for H_name, H_fn in lambdified_hamiltonians_B.items()
}



In [92]:
H_e = H["Hrot"]+H["H_mhf_Tl"]+H["H_mhf_F"]+H["H_c_Tl"]+H["H_LD"]+H["H_cp1_Tl"]+Ez*H["HSz"]+Bz*H["HZz"]
print(np.allclose(H_e, H_e.conj().T))
D_e, V_e = np.linalg.eigh(H_e)

True


In [93]:
excited_states_all = matrix_to_states(V_e,QN_e)
excited_states = excited_states_all[2*4*(3+1)**2-8-14-10-18:2*4*(3+1)**2-14-8-10]

In [94]:
for state in excited_states[:]:
    print("")
    state.print_state(tol=0.01)


+0.9566+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.1139+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.2663+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.0321+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

-0.1139+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.9566+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0321+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.2663+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

-0.9516+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.1495+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.2650+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 =

### Eigenstates for X

In [95]:
#Substitute values into hamiltonian
variables = [
    sympy.symbols('Brot'),
    *sympy.symbols('c1 c2 c3 c4'),
    sympy.symbols('D_TlF'),
    *sympy.symbols('mu_J mu_Tl mu_F')
]

lambdified_hamiltonians_X = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians_X.items()
}

#Molecular constants

# Values for rotational constant are from "Microwave Spectral tables: Diatomic molecules" by Lovas & Tiemann (1974). 
# Note that Brot differs from the one given by Ramsey by about 30 MHz.
B_e = 6.689873e9
c1 = 126030.0 
c2 = 17890.0
c3 = 700.0
c4 = -13300.0
alpha = 45.0843e6
Brot = B_e - alpha/2
D_TlF = 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]
mu_J = 35 # Hz/G
mu_Tl = 1240.5 #Hz/G
mu_F = 2003.63 #Hz/G

H_g = {
    H_name : H_fn(
        Brot,
        c1, c2, c3, c4,
        D_TlF,
        mu_J, mu_Tl, mu_F
    )
    for H_name, H_fn in lambdified_hamiltonians_X.items()
    }

In [96]:
H_g = H_g["Hff"]+Ez*H_g["HSz"]+Bz*H_g["HZz"]
print(np.allclose(H_g, H_g.conj().T))
D_g, V_g = np.linalg.eigh(H_g)

True


In [102]:
#Transform ground state eigenstates to coupled basis
ground_states_all = matrix_to_states(V_g,QN_g)
ground_states = []
for i, ground_state in enumerate(tqdm(ground_states_all[0:4*(4+1)**2])):
    ground_states.append(ground_state.remove_small_components(tol = 0.0001).transform_to_coupled())

In [103]:
for state in ground_states[:8]:
    print("")
    state.print_state(tol=0.01)


-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+1.0000+0.0000j x |J = 0, F1 = 1/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-1.0000+0.0000j x |J = 0, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-0.9429+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.3330+0.0000j x |J = 1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

+0.9190+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.3941+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

-0.9384+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.1017+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.3302+0.0000j x |J = 1, F1 = 3/2, F = 2, mF = 1, I1 = 1/2, I2 = 1/2, O

## Electric dipole matrix elements
Calculate the electric dipole matrix elements between uncoupled basis states in X and B state 

In [104]:
def ED_ME(ground_state, excited_state, pol_vec = np.array([0,0,1]), rme_only = False):
    """
    Function for calculating electric dipole matrix elements between UncoupledBasisStates.
    
    inputs:
    ground_state = UncoupledBasisState object in the X-state of TlF
    excited_state = UncoupledBasisState object in the B-state of TlF
    pol_vec = polarization vector for the light that is driving the transition
    rme_only = True if want to return reduced matrix element, False if want angular component also
    
    returns:
    ME = (reduced) electric dipole matrix element between ground_state and excited_state
    """
    
    #Find quantum numbers for ground state
    F = ground_state.F
    mF = ground_state.mF
    J = ground_state.J
    F1 = ground_state.F1
    I1 = ground_state.I1
    I2 = ground_state.I2
    Omega_g = ground_state.Omega
    
    #Find quantum numbers for excited state
    Fp = excited_state.F
    mFp = excited_state.mF
    Jp = excited_state.J
    F1p = excited_state.F1
    I1p = excited_state.I1
    I2p = excited_state.I2
    Omega_e = excited_state.Omega
    
    #Calculate the reduced matrix element
    q = Omega_g - Omega_e
    ME = ((-1)**(F1p+J+Fp+F1) * np.sqrt((2*F+1)*(2*Fp+1)*(2*F1p+1)*(2*F1+1)) * sixj_f(F1p,Fp,I2,F,F1,1) 
          * sixj_f(Jp,F1p,I1,F1,J,1) * (-1)**(J-Omega_g) *np.sqrt((2*J+1)*(2*Jp+1)) * threej_f(J,1,Jp,-Omega_g, q, Omega_e)
          * float(np.abs(q) < 2))
    
    #If we want the complete matrix element, calculate angular part
    if not rme_only:
        
        #Calculate elements of the polarization vector in spherical basis
        p_vec = {}
        p_vec[-1] = -1/np.sqrt(2) * (pol_vec[0] + 1j *pol_vec[1])
        p_vec[0] = pol_vec[2]
        p_vec[1] = +1/np.sqrt(2) * (pol_vec[0] - 1j *pol_vec[1])
        
        #Calculate the value of p that connects the states
        p = mF-mFp
        p = p*int(np.abs(p) <= 1)
        #Multiply RME by the angular part
        ME = ME * (-1)**(F-mF) * threej_f(F,1,Fp, -mF, p, mFp) * int(np.abs(p) <= 1)
    
    #Return the matrix element
    return ME
    
    
    

In [105]:
#Generate a matrix that contains the reduced electric dipole matrix elements between all excited and ground states of interest             
def generate_ME_matrix(H, QN_g=QN_g_coupled, QN_e = QN_e):
    #result = sympy.zeros(len(ground_states),len(excited_states))
    result = np.zeros((len(QN_g),len(QN_e)),dtype = complex)
    for i,a in tqdm(enumerate(QN_g)):
        for j,b in enumerate(QN_e):
            result[i,j] = H(a,b)
            
    return result

MEs = generate_ME_matrix(ED_ME)
MEs_states = V_g.conj().T @ MEs @ V_e

def calculate_branching_ratio(ground_state, excited_state, MEs = MEs, QN_g = QN_g_coupled, 
                              QN_e = QN_e):
    
    #Get the ground_state and excited_state state vectors
    vec_g = ground_state.state_vector(QN_g)
    vec_e = excited_state.state_vector(QN_e)
    
    #Find the reduced matrix element between ground_state and excited_state (for calculating partial width to ground_state)
    ME_ge = vec_g.conj().T @ MEs @ vec_e
    
    #Find the reduced matrix elements between the excited_state and all possible ground states
    #so the total width of the excited state can be calculated
    ME_e_all = V_g.conj().T @ MEs @ vec_e
    
    #Calculate branching ratio = partial width/total width
    branching_ratio = ME_ge*ME_ge.conj()/(ME_e_all.conj().T @ ME_e_all)
    
    return branching_ratio
    


In [106]:
#Loop over all excited states and calculate their branching ratios to all ground states
for excited_state in excited_states:
    print(10*"*")
    print("|e> =")
    excited_state.print_state()
    print("\nBranching ratios:")
    #Loop over ground states
    for ground_state in ground_states:
        br = np.real(calculate_branching_ratio(ground_state, excited_state))
        if br > 0.00005:
            print("{:5f} to \n|g> =".format(br))
            ground_state.print_state(tol = 0.01)
            print("")
        

**********
|e> =
+0.9566+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.1139+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.2663+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>

Branching ratios:
0.327705 to 
|g> =
-1.0000+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000103 to 
|g> =
-0.9992+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0352+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0176+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.005409 to 
|g> =
-0.0367+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9952+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0907+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 =

0.006027 to 
|g> =
-0.0367+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9952+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0907+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.002775 to 
|g> =
+0.0162+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0435+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9949+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0895+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.065653 to 
|g> =
-1.0000+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.142741 to 
|g> =
-0.0143+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0913+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P

0.002407 to 
|g> =
+0.0313+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9988+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0365+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.001834 to 
|g> =
-0.0330+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9987+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0380+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.001861 to 
|g> =
-0.0341+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9990+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0299+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.185391 to 
|g> =
+0.0366+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-

0.157994 to 
|g> =
+0.0385+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9992+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.119809 to 
|g> =
+0.0305+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9995+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.034311 to 
|g> =
-0.0168+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9999+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

**********
|e> =
+0.2150+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.9389+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.2615+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
0.000195 to 
|g> =
-0.9499+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 1,

0.000233 to 
|g> =
+0.9499+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.3028+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0711+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0313+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000208 to 
|g> =
-0.9975+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0270+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0659+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000119 to 
|g> =
+0.9996+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0273+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000308 to 
|g> =
-0.3078+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P =

...

0.000084 to 
|g> =
+0.9499+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.3028+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0711+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0313+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000281 to 
|g> =
-0.9975+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0270+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0659+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000110 to 
|g> =
-0.3078+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9493+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0523+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = -1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0360+0.0

0.000109 to 
|g> =
-0.3075+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9494+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0524+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0360+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.003658 to 
|g> =
+0.0415+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9724+0.0000j x |J = 2, F1 = 5/2, F = 2, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.2298+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.005308 to 
|g> =
+1.0000+0.0000j x |J = 2, F1 = 5/2, F = 3, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.004208 to 
|g> =
-0.0545+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0795+0.0000j x |J = 2, F1 = 3/2, F = 2, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+

0.000154 to 
|g> =
+0.1150+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9924+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.0401+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0152+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.013135 to 
|g> =
-0.0367+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9952+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0907+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.004598 to 
|g> =
+0.0162+0.0000j x |J = 3, F1 = 5/2, F = 2, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0435+0.0000j x |J = 3, F1 = 5/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+0.9949+0.0000j x |J = 3, F1 = 7/2, F = 3, mF = -2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0895+0.0

0.003823 to 
|g> =
+0.0447+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9985+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0324+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.002995 to 
|g> =
+0.0413+0.0000j x |J = 4, F1 = 7/2, F = 3, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9986+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0337+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 2, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.001568 to 
|g> =
+0.0313+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9988+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.0365+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>

0.000859 to 
|g> =
-0.0330+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
+

0.003128 to 
|g> =
+0.0382+0.0000j x |J = 4, F1 = 9/2, F = 4, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>
-0.9993+0.0000j x |J = 4, F1 = 9/2, F = 5, mF = 3, I1 = 1/2, I2 = 1/2, Omega = 0, P = None>



In [107]:
def branching_ratio_table(excited_state, ground_states):
    nominal_state = excited_state.find_largest_component()
    nominal_J = nominal_state.J
    nominal_F1 = nominal_state.F1
    nominal_F = nominal_state.F
    mF = 0
    I1 = 1/2
    I2 = 1/2
    print(10*"*")
    print("|e> =")
    excited_state.print_state()
    print("\nBranching ratios:")
    
    branching_ratios = {-3:0, -2:0,-1:0, 0:0, 1:0, 2:0, 3:0}
    
    for ground_state in ground_states:
        J_g = ground_state.find_largest_component().J
        DeltaJ = J_g-nominal_J
        
        if np.abs(DeltaJ) < 4:
            br = np.real(calculate_branching_ratio(ground_state, excited_state))
            branching_ratios[DeltaJ] += br
            
    for DeltaJ, BR in branching_ratios.items():
        print("{:d}: {:.5f}".format(DeltaJ, BR))
        

In [108]:
for excited_state in excited_states:
    branching_ratio_table(excited_state,ground_states)

**********
|e> =
+0.9566+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.1139+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.2663+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = -4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.00000
-1: 0.32770
0: 0.37012
1: 0.28997
2: 0.00000
3: 0.00000
**********
|e> =
-0.1139+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.9566+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
-0.2663+0.0000j x |J = 4, F1 = 7/2, F = 4, mF = 4, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

Branching ratios:
-3: 0.00000
-2: 0.00000
-1: 0.32770
0: 0.37012
1: 0.28997
2: 0.00000
3: 0.00000
**********
|e> =
-0.9516+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.1495+0.0000j x |J = 3, F1 = 7/2, F = 4, mF = -3, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.265